In [7]:
from lxml import html
from lxml import objectify
import requests
import json
import pandas as pd
import csv
import numpy as np
import Utils
from importlib import reload

In [18]:
class Scraper_BackCountry:
    def __init__(self):
        self.BASE_URL = "https://www.backcountry.com"
        self.HEADERS = {
            'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36",
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'referer': 'https://www.google.com/',
        }
    
    def getSpecs(self, url):
        try:
            tree = self.get(url)

            specNames = tree.xpath('//dt[@class="prod-details-accordion__techspec-name js-techspec-name"]/text()')
            specValues = tree.xpath('//dd[@class="prod-details-accordion__techspec-value js-techspec-value"]/text()')

            try:
                img = tree.xpath('//img[@class="ui-flexslider__img js-flexslider-img qa-flexslider__img"]')[0]
                img = html.tostring(img, encoding="unicode")
                img = img.split('data-src="//')[1].split('"')[0]
            except Exception as e:
                print(e)
                img = ""

            name = tree.xpath('//title/text()')[0].split(" |")[0]
            summary = tree.xpath('//script[@type="text/javascript"]/text()')[3]
            summary = summary.split('BC.product = ')[1].split(';BC.product')[0]
            summary = json.loads(summary)
            try:
                currency = tree.xpath('//meta[@itemprop="priceCurrency"]/@content')[0] # TODO fix currency for when out of stock
            except Exception as e:
                print(e)
                currency = 'USD'
            avgRating, numRatings = self.calculateRating(tree.xpath('//div[@class="reviews-wrap"]')[0])

            data = {}
            data["URL"] = url
            data["Name"] = summary["displayName"]
            data["Brand"] = summary["brand"]["displayName"]

            try:
                data["lowPrice"] = summary["lowListPrice"].split("$")[1]  #maybe add symbols for different currencies
            except Exception as e:
                print(e)
                data["lowPrice"] = -1

            try:
                data["highPrice"] = summary["highListPrice"].split("$")[1]
            except Exception as e:
                print(e)
                data["highPrice"] = -1

            try:
                data["PriceCurrency"] = currency # TODO fix currency for when out of stock
            except Exception as e:
                print(e)
                data["PriceCurrency"] = 'USD'
            data["Image"] = img
            data['Rating'] = avgRating
            data['numRatings'] = numRatings
            for i, item in enumerate(specValues):
                data[specNames[i]] = item
        except Exception as e:
            data = {}
            data["URL"] = url
            print("could not get data for " + url)
        return data

    
    def getPages(self, url):
        tree = self.get(url)
        numPagesList = tree.xpath('//li[@class="page-link page-number qa-page-link"]/.//a/text()')
        if(len(numPagesList) == 0):
            return [url]

        urlList = []
        numPages = int(numPagesList[-1])
        if "?" in url:
            pagesStr1 = "&page="
        else:
            pagesStr1 = "?page="
        pagesStr2 = "&pagesize=42"    # for some reason always added when changing page manually on website
        for i in range(numPages):
            pageURL = url + pagesStr1 + str(i) + pagesStr2
            urlList.append(pageURL)
        return urlList
    
    
    
    def getProducts(self, urls):
        urlPages = self.getPages(urls) # get urls of each page of the website category
        urlProducts = []
        for url in urlPages: #get urls of each product on each page and append to list
            productsURL = self.getProductsFromPage(url)
            for productURL in productsURL:
                urlProducts.append(productURL)
        products = []
        for prodURL in urlProducts:
            print(prodURL)
            specs = self.getSpecs(self.BASE_URL + prodURL)
            products.append(specs) # append product list to specs
        return products


    def scrape(self, categories):
        products = {}
        for category in categories:
            print(category)
            products[category] = []
            for url in categories[category]:
                print(url)
                products[category].append(self.getProducts(url))
            print("saving...")
            Utils.save_json(products)
            print("saved!")
        return products
    
    
    
    # calculates produt rating from reviews listed on product page
    def calculateRating(self, tree):
        #get number of reviews of each numbered rating
        numFive = len(tree.xpath('//div[@class="rating-value-5 review-header-rating"]'))
        numFour = len(tree.xpath('//div[@class="rating-value-4 review-header-rating"]'))
        numThree = len(tree.xpath('//div[@class="rating-value-3 review-header-rating"]'))
        numTwo = len(tree.xpath('//div[@class="rating-value-2 review-header-rating"]'))
        numOne = len(tree.xpath('//div[@class="rating-value-1 review-header-rating"]'))
        
        numSum = numFive + numFour + numThree + numTwo + numOne
        # return -1 if product has no ratings
        if(numSum == 0):
            return -1, 0
        #calculate average rating and total number of ratings
        return ( ( 5 * numFive + 4 * numFour + 3 * numThree + 2 * numTwo + 1 * numOne) / numSum ), numSum
    
    def getProductsFromPage(self, url):
        tree = self.get(url)

        products = tree.xpath('//div[@aria-label="Product"]/.//a/@href')
        return products
    
        # helper function for get requests
    def get(self, url):
        return Utils.get(url, self.HEADERS)

In [19]:
scraper = Scraper_BackCountry()
data = scraper.scrape(categories)
Utils.save_json(data)
print("done!")

Backpack
https://www.backcountry.com/technical-daypacks
/osprey-packs-talon-22-backpack-1200-1300cu-in?skid=OSP007Z-ULTBL-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MToxOmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/osprey-packs-tempest-30-backpack-womens-1709-1831cu-in-osp008d?skid=OSP008D-IRIBL-XSS&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MToyOmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/osprey-packs-talon-33-backpack-1900-2000cu-in-osp007y?skid=OSP007Y-ULTBL-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTozOmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/mountain-hardwear-scrambler-35l-backpack?skid=MHW01CQ-BKMUL-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTo0OmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/ortovox-peak-45-backpack-2750cu-in?skid=ORT003H-SAFBLU-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTo1OmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/patagonia-ascensionist-55l-backpack?skid=PATZ9HR-FIR-LXL&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTo2OmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/mystery-ranch-scree-backpack-1953cu-in?skid=MYT002Z-CAR-SM&ti=UExQIEN

/marmot-kompressor-meteor-16l-backpack?skid=MAR01R2-BLAGRE-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoxODpiYy10ZWNobmljYWwtZGF5cGFja3M=
/osprey-packs-kamber-32-backpack-1831-1953cu-in?skid=OSP006C-RIPRD-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoxOTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/mammut-trion-nordwand-28l-backpack-womens?skid=MAMU59C-SKYNIG-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/mammut-lithium-crest-s-30-plus-7-backpack-1830cu-in?skid=MAM01BY-GAL-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/mountain-hardwear-grotto-35l-backpack?skid=MHWZ9E1-ALD-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMjpiYy10ZWNobmljYWwtZGF5cGFja3M=
/osprey-packs-hikelite-18-backpack?skid=OSP00B0-KUMOR-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMzpiYy10ZWNobmljYWwtZGF5cGFja3M=
/burton-day-hiker-pack-bur01z2?skid=BUR01Z2-MAOLFLSA-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyNDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/

/arc-teryx-index-15-backpack?skid=ARC00HO-DARDAK-Y001&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MzozNTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/ortovox-peak-32-short-backpack-1950cu-in?skid=ORT003G-DARBLO-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MzozNjpiYy10ZWNobmljYWwtZGF5cGFja3M=
/patagonia-cragsmith-35l-backpack-2136cu-in?skid=PAT02HO-BAYBLU-LXL&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MzozNzpiYy10ZWNobmljYWwtZGF5cGFja3M=
/matador-freefly16-backpack?skid=MTU000L-CHA-S16LIT&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MzozODpiYy10ZWNobmljYWwtZGF5cGFja3M=
/ortlieb-gear-pack-40l?skid=OLB001B-BLA-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MzozOTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/gregory-sula-28-backpack-womens-1708cu-in?skid=GRE009N-NIGGRE-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6Mzo0MDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/gregory-juno-h20-24l-daypack-womens?skid=GREZ062-LAGBLU-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6Mzo0MTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/gregory-inertia-20-backpack?skid=GRE009R-FE

/burton-beeracuda-gearhaus-42l-cooler-bag?skid=BURZ897-TRUBLA-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NToxMTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/deuter-speed-lite-32l-backpack-dtr006u?skid=DTR006U-BK-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NToxMjpiYy10ZWNobmljYWwtZGF5cGFja3M=
/marmot-v10-pack?skid=MARZ96D-ESTBLUECL-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NToxMzpiYy10ZWNobmljYWwtZGF5cGFja3M=
/gregory-inertia-30-backpack?skid=GRE009P-ESTBL-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NToxNDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/fjallraven-singi-20-backpack?skid=FJR00R9-CABRED-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NToxNTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/gregory-juno-36l-daypack-womens?skid=GREZ05Y-SPRBLU-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NToxNjpiYy10ZWNobmljYWwtZGF5cGFja3M=
/fjallraven-keb-hike-20-backpack?skid=FJR00ON-OLIFO-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NToxNzpiYy10ZWNobmljYWwtZGF5cGFja3M=
/matador-freerain-24l-backpack?skid=MTU000O-COY-S24LIT&ti=

/osprey-packs-archeon-30l-backpack?skid=OSPZ184-STABLU-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NjoyODpiYy10ZWNobmljYWwtZGF5cGFja3M=
/black-diamond-distance-8-backpack?skid=BLD00YL-BK-M&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NjoyOTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/black-diamond-dawn-patrol-15-backpack-915cu-in?skid=BLD00E2-DEETOR-ML&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NjozMDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/thule-capstone-32-backpack-1952cu-in?skid=THU004Y-OBS-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NjozMTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/black-diamond-cirque-45-backpack-2746cu-in?skid=BLD00L4-BK-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NjozMjpiYy10ZWNobmljYWwtZGF5cGFja3M=
/black-diamond-street-creek-20l-backpack?skid=BLD00YN-BK-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NjozMzpiYy10ZWNobmljYWwtZGF5cGFja3M=
/deuter-attack-20-backpack-1220cu-in?skid=DTR004G-BK-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6NjozNDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/deuter-trans-alpine-28-sl-1709cu

...

/mountain-hardwear-bozeman-15-sleeping-bag-15f-synthetic?skid=MHWZ998-CITSUN-LONRIGZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjE6Mjg6YmMtc2xlZXBpbmctYmFncw==
/marmot-fulcrum-15-sleeping-bag-15-degree-down?skid=MAR01YV-VNNAVIND-REGZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjE6Mjk6YmMtc2xlZXBpbmctYmFncw==
/marmot-trestles-elite-eco-0-sleeping-bag-0-degree-synthetic?skid=MAR01RJ-ORHAZSTE-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjE6MzA6YmMtc2xlZXBpbmctYmFncw==
/alps-mountaineering-crescent-lake-sleeping-bag-20-degree-synthetic?skid=ALM0091-KIW-REG&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjE6MzE6YmMtc2xlZXBpbmctYmFncw==
/kelty-callisto-30-sleeping-bag-30-degree-synthetic-womens?skid=KEL007R-MAL-REGZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjE6MzI6YmMtc2xlZXBpbmctYmFncw==
/marmot-trestles-30-sleeping-bag-30-degree-synthetic?skid=MAR013N-GNLIC-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjE6MzM6YmMtc2xlZXBpbmctYmFncw==
/kelty-tuck-sleeping-bag-0-degree-synthetic?skid=KELO07Y-BEL-LONLEFZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjE6MzQ6Ym

/alps-mountaineering-razor-fleece-sleeping-bag-liner?skid=ALM006N-RUS-ONESIZ&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjM6MTpiYy1zbGVlcGluZy1iYWdz
/marmot-nanowave-50-semi-rec-sleeping-bag-50-degree-synthetic?skid=MAR01RN-ESTBL-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjM6MjpiYy1zbGVlcGluZy1iYWdz
/big-agnes-lost-dog-sleeping-bag-45-degree-synthetic?skid=BAG00EZ-ORA-REGZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjM6MzpiYy1zbGVlcGluZy1iYWdz
/big-agnes-big-creek-sleeping-bag-30-degree-synthetic?skid=BAG00F6-YEL-DOU&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjM6NDpiYy1zbGVlcGluZy1iYWdz
/marmot-ultra-elite-30-sleeping-bag-30-degree-synthetic?skid=MAR01BN-DARSTEGN-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjM6NTpiYy1zbGVlcGluZy1iYWdz
/therm-a-rest-saros-sleeping-bag-0f-synthetic?skid=TAEA061-STA-SMA&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjM6NjpiYy1zbGVlcGluZy1iYWdz
/nemo-equipment-inc.-jazz-sleeping-bag-20-degree-synthetic?skid=NEM0083-ONECOL-REG&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjM6NzpiYy1zbGVlcGluZy1iYWdz
/kelty-mistral-sleeping-bag-

/therm-a-rest-space-cowboy-sleeping-bag-45-degree-synthetic?skid=CAS00DP-ETH-LON&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjQ6MTc6YmMtc2xlZXBpbmctYmFncw==
/marmot-paiju-sleeping-bag-5f-down?skid=MARZ96A-BLA-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjQ6MTg6YmMtc2xlZXBpbmctYmFncw==
/marmot-cwm-eq-sleeping-bag-40-degree-down?skid=MAR00IA-TEARED-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjQ6MTk6YmMtc2xlZXBpbmctYmFncw==
/sea-to-summit-spark-spii-sleeping-bag-35-degree-down?skid=STS006Z-ONECOL-REG&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjQ6MjA6YmMtc2xlZXBpbmctYmFncw==
/kelty-rambler-50-sleeping-bag-50-degree-synthetic?skid=KEL007O-RDSD-REGZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjQ6MjE6YmMtc2xlZXBpbmctYmFncw==
/big-agnes-oliphant-sleeping-bag-0-degree-down?skid=BAG00FF-BLU-PETZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjQ6MjI6YmMtc2xlZXBpbmctYmFncw==
/therm-a-rest-vesper-sleeping-bag-20-degree-down?skid=CAS00ER-VAP-REG&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjQ6MjM6YmMtc2xlZXBpbmctYmFncw==
/marmot-teton-sleeping-bag-0-degree-down-womens

/big-agnes-anvil-horn-sleeping-bag-45-degree-down?skid=BAG00ET-BLU-REGZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjU6MzM6YmMtc2xlZXBpbmctYmFncw==
/nemo-equipment-inc.-tempo-35-sleeping-bag-35f-synthetic?skid=NEMK014-SAPDARTIM-REG&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjU6MzQ6YmMtc2xlZXBpbmctYmFncw==
/big-agnes-torchlight-ul-sleeping-bag-30f-down?skid=BAGZ28F-ORA-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjU6MzU6YmMtc2xlZXBpbmctYmFncw==
/mountain-hardwear-phantom-sleeping-bag-15-degree-down?skid=MHW01CX-ALPRD-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjU6MzY6YmMtc2xlZXBpbmctYmFncw==
/western-mountaineering-alder-mf-sleeping-bag-25-degree-down?skid=WES000J-ONECOL-S6FTLEFTZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjU6Mzc6YmMtc2xlZXBpbmctYmFncw==
/western-mountaineering-apache-gore-windstopper-sleeping-bag-15-degree-down?skid=WES000Q-ONECOL-S5FTS6INRIGHT&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjU6Mzg6YmMtc2xlZXBpbmctYmFncw==
/marmot-micron-50-sleeping-bag-50-degree-down?skid=MAR01BC-CROSTO-REGZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWd

/western-mountaineering-apache-mf-sleeping-bag-15-degree-down?skid=WES0043-ONECOL-S6FTLEFTZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjc6NjpiYy1zbGVlcGluZy1iYWdz
/kelty-stardust-sleeping-bag-15-degree-synthetic?skid=KEL00AH-REFPON-LONZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjc6NzpiYy1zbGVlcGluZy1iYWdz
/the-north-face-eco-trail-sleeping-bag-35f-synthetic?skid=TNFZ9TZ-TNFYELHEM-REGLEFZIP&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjc6ODpiYy1zbGVlcGluZy1iYWdz
/therm-a-rest-oberon-sleeing-bag-0-degree-down?skid=CAS00DN-ELE-LON&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjc6OTpiYy1zbGVlcGluZy1iYWdz
/nemo-equipment-inc.-disco-30-sleeping-bag-30f-down-womens?skid=NEMK00Y-CELMOO-REG&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjc6MTA6YmMtc2xlZXBpbmctYmFncw==
/therm-a-rest-saros-sleeping-bag-32f-synthetic?skid=TAEA062-STA-LON&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjc6MTE6YmMtc2xlZXBpbmctYmFncw==
/kelty-cosmic-sleeping-bag-40f-synthetic?skid=KELO07X-DUFBAGSMO-LON&ti=UExQIENhdDpTbGVlcGluZyBCYWdzOjc6MTI6YmMtc2xlZXBpbmctYmFncw==
/rab-expedition-1000-slee

In [9]:
categories = {}

backPackURLs = [
    'https://www.backcountry.com/technical-daypacks',
    'https://www.backcountry.com/hydration-packs',
    'https://www.backcountry.com/backpacking-packs',
    'https://www.backcountry.com/kid-carriers'
]

tentURLS = [
    'https://www.backcountry.com/3-season-backpacking-tents',
    'https://www.backcountry.com/4-season-tents',
    
]

bivyURLs = [
    'https://www.backcountry.com/bivys'
]

sleepingBagURLs = [
    'https://www.backcountry.com/sleeping-bags',   
]

categories['Backpack'] = backPackURLs
categories['Tent'] = tentURLS
categories['Bivy'] = bivyURLs
categories['Sleeping Bag'] = sleepingBagURLs